# Сервис для коспектирования медицинских статей по разделам: "RESULTS", "CONCLUSIONS", "KEY TERMS", "REFERENCES" 

In [65]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="spacy.util")
warnings.filterwarnings(
    action="ignore",
    message=r"Upper case characters found in vocabulary while 'lowercase' is True",
    category=UserWarning,
    module=r"sklearn\.feature_extraction\.text"
)

In [48]:
import torch
from transformers import LEDTokenizer, LEDForConditionalGeneration, AutoTokenizer, AutoModel
from keybert import KeyBERT
import spacy
import gradio as gr
from sklearn.cluster import AgglomerativeClustering
import pandas as pd
import os
import re

In [49]:
import torch
print(torch.__version__)

2.1.0+cu118


In [50]:
import transformers
print(transformers.__version__)

4.54.0.dev0


In [51]:
def load_models():
    device = "cuda"
    pubmed_model = AutoModel.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract", use_safetensors=True).to(device)
    
    return {
        "led": {
            "tokenizer": LEDTokenizer.from_pretrained("./led_summarization"),
            "model": LEDForConditionalGeneration.from_pretrained("./led_summarization", use_safetensors=True).to(device)
        },
        "pubmed": {
            "tokenizer": AutoTokenizer.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract"),
            "model": pubmed_model
        },
        "keybert": KeyBERT(model=pubmed_model),
        "spacy": spacy.load("en_core_sci_sm") 
    }

In [52]:
models = load_models()
nlp = models["spacy"]  
kw_model = models["keybert"]  
pubmed_tokenizer = models["pubmed"]["tokenizer"]  
led_model = models["led"]["model"]  
led_tokenizer = models["led"]["tokenizer"]

Генерация и форматирование (format_medical_summary) разделов: "RESULTS", "CONCLUSIONS"

In [53]:
def generate_summary(medical_text):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    inputs = led_tokenizer(
        medical_text,
        max_length=4096,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    ).to(device)
    
    with torch.no_grad():
        outputs = led_model.generate(
            input_ids=inputs["input_ids"],
            max_length=256,
            num_beams=4,
            early_stopping=True,
            length_penalty=1.2,
            no_repeat_ngram_size=3,
            repetition_penalty=1.5
        )
        generated_summary = led_tokenizer.decode(outputs[0], skip_special_tokens=False)
    
    return format_medical_summary(generated_summary)

In [54]:
def format_medical_summary(generated_text):
    clean_text = generated_text.replace('</s>', '').replace('<s>', '').strip()
    results_section = ''
    conclusions_section = ''
    
    if '<results>' in clean_text:
        results_part = clean_text.split('<results>')[1]
        results_section = results_part.split('<conclusions>')[0].strip()
    
    if '<conclusions>' in clean_text:
        conclusions_part = clean_text.split('<conclusions>')[1]
        conclusions_section = conclusions_part.split('<dig>')[0].strip()
    
    formatted_output = ""
    if results_section:
        results_section = results_section[0].upper() + results_section[1:]
        formatted_output += "RESULTS:\n" + results_section + "\n\n"
    
    if conclusions_section:
        conclusions_section = conclusions_section[0].upper() + conclusions_section[1:]
        formatted_output += "CONCLUSIONS:\n" + conclusions_section
    
    return formatted_output.strip()

Разбиение длинного текста на меньшие фрагменты для обработки моделями (PubMedBERT, en_core_sci_sm) с ограничением на длину входа

In [55]:
def chunk_text(text, max_tokens=512, stride=128):
    sentences = [sent.text for sent in nlp(text).sents]
    current_chunk = []
    current_length = 0
    chunks = []
    
    for sentence in sentences:
        sent_tokens = pubmed_tokenizer.tokenize(sentence)
        if current_length + len(sent_tokens) > max_tokens:
            chunks.append(" ".join(current_chunk))
            current_chunk = current_chunk[-stride//2:]  
            current_length = len(current_chunk)
        current_chunk.append(sentence)
        current_length += len(sent_tokens)
    
    if current_chunk:
        chunks.append(" ".join(current_chunk))
    return chunks

Извлечение потенциальных ключевых фраз:
- Именные группы (1-5 слов)
- Пары "Прилагательное/Существительное + Существительное"
- Аббревиатуры (2-6 заглавных букв)

In [56]:
def extract_candidates(text):
    doc = nlp(text)
    noun_chunks = {        
        " ".join(tok.text for tok in chunk).lower()
        for chunk in doc.noun_chunks if 1 <= len(chunk) <= 5
    }
    extras = {
        f"{doc[i].text} {doc[i+1].text}".lower()
        for i in range(len(doc)-1)
        if doc[i].pos_ in {"ADJ","NOUN"} and doc[i+1].pos_=="NOUN"
    }
    abbrs = {t.text for t in doc if t.is_upper and 2<=len(t.text)<=6}
    return list(noun_chunks | extras | abbrs)

Получение кандидатов через extract_candidates() + извлечение фраз длиной 2-5 слов, применение MMR (Maximal Marginal Relevance) для разнообразия

In [57]:
def extract_keyphrases(text, top_n=30):
    kw = kw_model.extract_keywords(
        text,
        candidates=extract_candidates(text),
        keyphrase_ngram_range=(2,5), 
        nr_candidates=80,          
        use_mmr=True, diversity=0.85, top_n=top_n*2
    )
    return kw[:top_n]

Группировка семантически схожих ключевых фразы с помощью агломеративной кластеризации с косинусной метрикой. Для каждого кластера выбирается фраза с наивысшим score

In [58]:
def group_similar(keywords, thresh=0.85):
    phrases = [p for p,_ in keywords]
    emb = kw_model.model.embed(phrases)  
    labels = AgglomerativeClustering(n_clusters=None,
                                     distance_threshold=1-thresh,
                                     affinity="cosine",
                                     linkage="average").fit_predict(emb)
    best = {}
    for (ph,sc),lb in zip(keywords,labels):
        if lb not in best or sc>best[lb][1]:
            best[lb]=(ph,sc)
    return sorted(best.values(), key=lambda x:x[1], reverse=True)

In [59]:
def extract_keyphrases_from_long_text(text):
    chunks = chunk_text(text)
    all_keywords = []
    for chunk in chunks:
        keywords = extract_keyphrases(chunk)
        all_keywords.extend(keywords)

    unique_keywords = {}
    for phrase, score in all_keywords:
        if phrase not in unique_keywords or score > unique_keywords[phrase]:
            unique_keywords[phrase] = score

    sorted_keywords = sorted(unique_keywords.items(), key=lambda x: x[1], reverse=True)
    return sorted_keywords[:30]

In [60]:
def format_keyterms_output(keywords):
    output = "KEY TERMS:\n"
    key_phrases = [f"- {phrase}" for phrase, score in keywords]
    output += "\n".join(key_phrases)
    return output

Функция extract_references извлекает из текста следующие типы ссылок и идентификаторов:
- Веб-адреса - распознаёт стандартные интернет-ссылки, включая URL с протоколами http/https (например, https://site.com, http://example.org) и адреса с префиксом www (www.research.edu)
- Научные публикации - находит идентификаторы научных работ: препринты arXiv в формате arxiv:1710.05006, цифровые идентификаторы DOI (как с префиксом doi:10.1016/j.cell.2021.01.001, так и без него 10.1001/jama.2020.1245)
- Медицинские базы данных - выявляет специализированные медицинские идентификаторы: PubMed ID (PMID: 12345678), PubMed Central ID (PMCID: PMC9876543), а также номера клинических испытаний в формате NCT01234567
- Библиографические данные - распознаёт международные стандартные номера книг ISBN в различных форматах записи, включая ISBN-13 (ISBN 978-3-16-148410-0, ISBN-13: 978-1234567890)
- Контактная информация - обнаруживает электронные адреса (author@university.edu)

In [61]:
def extract_references(text):
    patterns = [
        r'(https?://[^\s<>"]+|www\.[^\s<>"]+)',                                # URL
        r'(arxiv:\d{4}\.\d{4,5})',                                             # arXiv
        r'doi:\s*10\.\d{4,9}/[-._;()/:A-Za-z0-9]+',                            # DOI с префиксом
        r'10\.\d{4,9}/[-._;()/:A-Za-z0-9]+',                                   # DOI без префикса
        r'PMID:\s*\d+',                                                        # PMID
        r'PMCID:\s*PMC\d+',                                                    # PMCID
        r'NCT\d{8}',                                                           # ClinicalTrials.gov
        r'ISBN(?:-13)?:?\s*(?:97[89][- ]?)?\d{1,5}[- ]?\d+[- ]?\d+[- ]?[\dX]', # ISBN
        r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}',                     # Email
    ]
    results = []
    for pattern in patterns:
        results.extend(re.findall(pattern, text, re.IGNORECASE))
    return results

In [62]:
def format_references_output(references):
    output = "REFERENCES:\n"
    references = [f"- {ref}" for ref in references]
    output += "\n".join(references)
    return output

In [63]:
def gradio_summarize(medical_text):
    if len(medical_text) < 3000:
        return "Пожалуйста, введите медицинский текст для не менее 3000 символов"
    summary = generate_summary(medical_text)
    keywords = extract_keyphrases_from_long_text(medical_text)
    formatted_output = format_keyterms_output(keywords)
    references = extract_references(medical_text)
    if len(references) != 0:
        return summary + '\n\n' + formatted_output + '\n\n' + format_references_output(references)
    
    return summary + '\n\n' + formatted_output

In [66]:
with gr.Blocks() as demo:
    gr.Markdown("# Автоматическое резюмирование медицинских публикаций: извлечение результатов, выводов, ключевых терминов, используемой литературы")
    gr.Markdown("Введите медицинский текст на английском языке (не менее чем 3000 символов). Модель сгенерирует краткий конспект по разделам.")

    input_text = gr.Textbox(
        lines=15,
        placeholder="Введите медицинский текст на английском языке...",
        label="Входной текст",
        elem_id="input-textbox"
    )

    output_text = gr.Textbox(
        lines=10,
        label="Конспет",
        interactive=False,
        elem_id="output-textbox"
    )

    summarize_btn = gr.Button("Сгенерировать конспект")

    summarize_btn.click(fn=gradio_summarize, inputs=input_text, outputs=output_text)

demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7864
IMPORTANT: You are using gradio version 3.41.2, however version 4.44.1 is available, please upgrade.
--------
Running on public URL: https://b7292743ad5946fa38.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
